# Clustering Crypto

In [386]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from pathlib import Path
import plotly.express as px

### Fetching Cryptocurrency Data

In [387]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [388]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

In [389]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")
crypto_df=pd.read_csv(file_path)
crypto_df.head()
# Create a DataFrame

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [390]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df2=crypto_df.drop(["Unnamed: 0"], axis=1)

In [391]:
crypto_df2.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [392]:
# Keep only cryptocurrencies that are trading
crypto_active=crypto_df2[(crypto_df2.IsTrading==True)]


In [393]:
crypto_active

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
1243,Super Zero,Ethash,True,PoW,NaN,1000000000
1244,UOS,SHA-256,True,DPoI,NaN,1000000000
1245,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [394]:
# Keep only cryptocurrencies with a working algorithm
algo_works=crypto_active.dropna()

In [395]:
algo_works

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
...,...,...,...,...,...,...
1238,ZEPHYR,SHA-256,True,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,True,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [396]:
# Remove the "IsTrading" column
algo_works=algo_works.drop(['IsTrading'], axis=1)

In [397]:
algo_works.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [398]:
# Remove rows with at least 1 null value
algo_works=algo_works.dropna()

In [399]:
# Remove rows with cryptocurrencies having no coins mined
algo_works=algo_works[(algo_works.TotalCoinsMined!=0)]

In [400]:
algo_works

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [401]:
# Drop rows where there are 'N/A' text values

new_crypto = algo_works[algo_works.apply(lambda x: x["CoinName"] == 'N/A' or x["Algorithm"] == 'N/A'or x["ProofType"] == 'N/A' or x["TotalCoinsMined"] == 'N/A' or x["TotalCoinSupply"] == 'N/A', axis=1)]

In [402]:
new_crypto

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply


In [403]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_df=pd.DataFrame(algo_works['CoinName'])
coin_df.head()

,CoinName
0,42 Coin
2,404Coin
5,EliteCoin
7,Bitcoin
8,Ethereum


In [404]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
algo_works=algo_works.drop(columns=['CoinName'])

In [405]:
algo_works.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 533 entries, 0 to 1247
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Algorithm        533 non-null    object 
 1   ProofType        533 non-null    object 
 2   TotalCoinsMined  533 non-null    float64
 3   TotalCoinSupply  533 non-null    object 
dtypes: float64(1), object(3)
memory usage: 20.8+ KB


In [406]:
algo_works.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,Scrypt,PoW/PoS,4.199995e+01,42
2,Scrypt,PoW/PoS,1.055185e+09,532000000
5,X13,PoW/PoS,2.927942e+10,314159265359
7,SHA-256,PoW,1.792718e+07,21000000
8,Ethash,PoW,1.076842e+08,0


In [407]:
# Create dummy variables for text features
crypto_enc = pd.get_dummies(algo_works)
crypto_enc.head()


,TotalCoinsMined,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,...,TotalCoinSupply_91388946,TotalCoinSupply_92000000000,TotalCoinSupply_9354000,TotalCoinSupply_9507271,TotalCoinSupply_9736000,TotalCoinSupply_98000000,TotalCoinSupply_98100000000,TotalCoinSupply_990000000000,TotalCoinSupply_999481516,TotalCoinSupply_9999999
0,4.199995e+01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.055185e+09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2.927942e+10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1.792718e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [408]:
# Standardize data
crypto_scaled = StandardScaler().fit_transform(crypto_enc)

In [409]:
print(crypto_scaled[0:5])

[[-0.11674788 -0.0433555  -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 [-0.09358885 -0.0433555  -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 [ 0.52587231 -0.0433555  -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 [-0.11635442 -0.0433555  -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]
 [-0.11438445 -0.0433555  -0.0433555  ... -0.0433555  -0.0433555
  -0.0433555 ]]


### Reducing Dimensions Using PCA

In [410]:
# Use PCA to reduce dimensions to 3 principal components
pca=PCA(n_components=4)

In [411]:
# Create a DataFrame with the principal components data
crypto_pca=pca.fit_transform(crypto_scaled)
df_crypto_pca=pd.DataFrame(
    data=crypto_pca, columns=["Principal component 1", "Principal component 2", " Prinipal component 3", "Principla component 4"]
)


In [412]:
df_crypto_pca.head()

,Principal component 1,Principal component 2,Prinipal component 3,Principla component 4
0,-0.406419,-0.106423,-0.028281,0.009874
1,-0.384391,-0.111723,-0.035184,0.017997
2,0.362762,-0.061712,-0.110975,-0.256142
3,-0.159569,-0.213919,0.002977,-0.099205
4,-0.195106,-0.203098,0.128440,0.014565


In [413]:
pca.explained_variance_ratio_

array([0.00831781, 0.00793505, 0.0079228 , 0.00788887])

### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [414]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_crypto_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [415]:
#optimal k=6
# Initialize the K-Means model
model = KMeans(n_clusters=6, random_state=0)
# Fit the model
model.fit(df_crypto_pca)
# Predict clusters
predictions = model.predict(df_crypto_pca)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
df_crypto_pca["class"]=model.labels_
df_crypto_pca.head()

,Principal component 1,Principal component 2,Prinipal component 3,Principla component 4,class
0,-0.406419,-0.106423,-0.028281,0.009874,0
1,-0.384391,-0.111723,-0.035184,0.017997,0
2,0.362762,-0.061712,-0.110975,-0.256142,0
3,-0.159569,-0.213919,0.002977,-0.099205,0
4,-0.195106,-0.203098,0.128440,0.014565,0


### Visualizing Results

#### 3D-Scatter with Clusters

In [416]:
# Create a 3D-Scatter with the PCA data and the clusters
fig=px.scatter_3d(
    df_crypto_pca,
    x="Principal component 3",
    y="Principal component 2",
    z="principla component 1",
    color=["class"],
    symbol="class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['Principal component 1', 'Principal component 2', ' Prinipal component 3', 'Principla component 4', 'class'] but received: Principal component 3

#### Table of Tradable Cryptocurrencies

In [ ]:
# Table with tradable cryptos


In [ ]:
# Print the total number of tradable cryptocurrencies


#### Scatter Plot with Tradable Cryptocurrencies

In [ ]:
# Scale data to create the scatter plot


In [ ]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
df_crypto_pca.hvplot.scatter(
    x="TotalCoinMined",
    y="TotalCoinSupply",
    hover_cols=["prediction cluster"],
    by="prediction cluster",
)